In [1]:
%store -r city_coord
%store -r air_coord
%store -r office_info
%store -r office_coords
%store -r of_size
%store -r of_ref

import mapping_functions as M
import ranking_functions as R
import location_functions as L

import os
import re
import pandas as pd
from dotenv import load_dotenv
from pymongo import MongoClient
from bs4 import BeautifulSoup
import geocoder
import requests
import folium
from folium.plugins import MousePosition
from folium.plugins import Draw
from geopy.distance import distance as getDistance
from IPython.display import HTML
load_dotenv()


dbName = 'companies'

db,start = L.connectCollection(dbName,'startups')
db,old_comp = L.connectCollection(dbName,'old_comp')
db,starbucks = L.connectCollection(dbName,'starbucks')
db,schools = L.connectCollection(dbName,'schools')
db,night = L.connectCollection(dbName,'night')
db,vegan = L.connectCollection(dbName,'vegan')

In [2]:
names = ["Old companies","Starbucks","Night bars","Schools","Startups","Vegan restaurant","Airport","Offices"]  
collections = [old_comp,starbucks,night,schools,start,vegan]
colors = ["red","darkgreen","purple","lightblue","green","orange"]

In [4]:
M.mapResults(city_coord,13,names,colors,collections,air_coord,office_coords,"initial_map")

In [5]:
weight = []
for n in names[:6]:
    weight.append(int(input(f"Introduce the weight of {n} (1-5): ")))

Introduce the weight of Old companies (1-5): 5
Introduce the weight of Starbucks (1-5): 5
Introduce the weight of Night bars (1-5): 5
Introduce the weight of Schools (1-5): 5
Introduce the weight of Startups (1-5): 5
Introduce the weight of Vegan restaurant (1-5): 5


In [6]:
points,adresses,air_dist = R.pointsFunction(collections,office_coords,office_info,air_coord,weight)

Creamos un nuevo Dataframe con la informacion a cerca de cada una de las oficinas donde podriamos potencialmente colocar la empresa:
    - Direccion
    - Tamaño de la oficina
    - Puntuacion en el ranking total
    - Distancia a startups, starbucks, compañias antiguas, aeropuerto, colegios y bares mas cercanos
    - Coordenadas
    - Referencia

In [7]:
url_1 = "https://42floors.com/"

df_offices = pd.DataFrame(columns=['Adress','Square Feet','Punctuation','Startups','Starbucks',
                                   'Old companies','Schools','Night bars','Vegan Rest','Airport','Location','Reference'])
 
for index in range(len(points)):
    total = sum(points[index][1:])-points[index][0]
    df_offices = df_offices.append({'Adress':adresses[index],'Punctuation':total,
                                    'Starbucks':points[index][1],'Schools':points[index][3],'Startups':points[index][4],
                                    'Old companies':-points[index][0],'Night bars':points[index][2],
                                    'Square Feet':of_size[index],'Location':office_coords[index],
                                   'Reference':url_1+of_ref[index],'Airport':air_dist[index],'Vegan Rest':points[index][5]},ignore_index=True)

In [8]:
df = df_offices.sort_values(by=['Punctuation'],ascending=False)
df.to_csv('../output/office-location.csv')
df = df.reset_index(drop=True)
df

,Adress,Square Feet,Punctuation,Startups,Starbucks,Old companies,Schools,Night bars,Vegan Rest,Airport,Location,Reference
0,"40 Court St, Boston, MA 02108, USA","4,000",40.00,12.50,12.5,-12.50,2.5,12.50,12.50,4.16538866183103 km,"[-71.0592945, 42.3588496]",https://42floors.com//us/ma/boston/40-court-st...
1,"164 Newbury St, Boston, MA 02116, USA","1,400",40.00,7.50,10.0,-12.50,12.5,10.00,12.50,5.931335598372138 km,"[-71.0785848, 42.3504322]",https://42floors.com//us/ma/boston/164-newbury...
2,"30 Kneeland St, Boston, MA 02111, USA",880,40.00,10.00,10.0,-11.25,7.5,11.25,12.50,4.6111167651759235 km,"[-71.06187899999999, 42.3508642]",https://42floors.com//us/ma/boston/30-kneeland...
3,"65 Harrison Ave, Boston, MA 02111, USA",530,38.75,11.25,10.0,-11.25,7.5,10.00,11.25,4.557701136401338 km,"[-71.0614302, 42.3513453]",https://42floors.com//us/ma/boston/65-harrison...
4,"420 Boylston St, Boston, MA 02116, USA","3,920",38.75,11.25,12.5,-11.25,7.5,6.25,12.50,5.4344974428186426 km,"[-71.072549, 42.351083]",https://42floors.com//us/ma/boston/420-boylsto...
...,...,...,...,...,...,...,...,...,...,...,...,...
63,"300 Congress St, Quincy, MA 02169, USA","10,600 - 10,770",0.00,0.00,0.0,-0.00,0.0,0.00,0.00,14.567636185360263 km,"[-71.0150002, 42.2345289]",https://42floors.com//us/ma/boston/300-congres...
64,"70 Franklin St, New York, NY 10013, USA","10,050 - 11,430",0.00,0.00,0.0,-0.00,0.0,0.00,0.00,309.6816058391006 km,"[-74.004108, 40.7179659]",https://42floors.com//us/ma/boston/70-franklin...
65,"37 Charles St, New York, NY 10014, USA",880,0.00,0.00,0.0,-0.00,0.0,0.00,0.00,308.441410835834 km,"[-74.0027379, 40.73511389999999]",https://42floors.com//us/ma/boston/37-charles-...
66,"345 Carroll St, Brooklyn, NY 11231, USA","6,800",0.00,0.00,0.0,-0.00,0.0,0.00,0.00,311.43017459278934 km,"[-73.9910116, 40.67919089999999]",https://42floors.com//us/ma/boston/345-a-st?li...


In [9]:
coordinates = int(input("Introduce the index of the office you want:" ))
selected_office = df["Location"].iloc[coordinates][::-1]

Introduce the index of the office you want:0


In [10]:
M.mapResults(selected_office,16,names,colors,collections,air_coord,selected_office,"final_map")

In [12]:
map_name = input("Which map do you want to see? (initial_map // final_map): ")


Which map do you want to see? (initial_map // final_map): initial_map


In [13]:
HTML(f'<iframe src=../output/{map_name}.html width=700 height=450></iframe>')